In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
import pandas as pd, numpy as np
df = pd.read_csv("/kaggle/input/playground-series-s5e2/train.csv")
print("Original Data shape",df.shape)
df.head()

Original Data shape (300000, 11)


,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [4]:
COLS = ['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof', 'Style', 'Color']
for c in COLS:
    nans = df[c].isna()
    df[c],_ = pd.factorize(df[c])
    df[c] = df[c].astype("float32")
    df.loc[nans,c] = np.nan
COLS2 = ['Compartments', 'Weight Capacity (kg)', 'Price']
for c in COLS2:
    df[c] = df[c].astype("float32")
print("Data after label encoding...")
df.head()

Data after label encoding...


,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,11.611723,112.158752
1,1,0.0,1.0,1.0,10.0,0.0,1.0,1.0,1.0,27.078537,68.880562
2,2,1.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0,16.643761,39.173199
3,3,2.0,2.0,1.0,8.0,0.0,0.0,1.0,1.0,12.937221,80.607933
4,4,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,17.749338,86.023117


In [5]:
df.drop('id',axis=1,inplace=True)

In [6]:
train = df.iloc[:290000]
valid = df.iloc[290000:]
print("Original subset train shape",train.shape)
print("Original subset valid shape",valid.shape)

Original subset train shape (290000, 10)
Original subset valid shape (10000, 10)


In [7]:
X_train = train[COLS+['Compartments', 'Weight Capacity (kg)']].values
X_train[:,-2] /= 2.0
X_train[:,-1] /= 2.0
y_train = train['Price'].values

X_valid = valid[COLS+['Compartments', 'Weight Capacity (kg)']].values
X_valid[:,-2] /= 2.0
X_valid[:,-1] /= 2.0
y_valid = valid['Price'].values

print(X_train.shape, X_valid.shape)

(290000, 9) (10000, 9)


In [11]:
X_train_n = np.sum(X_train**2, axis=1,keepdims=True)
X_valid_n = np.sum(X_valid**2, axis=1,keepdims=True)
X_train_n.shape, X_valid_n.shape

((290000, 1), (10000, 1))

In [14]:
dist = X_valid_n - np.dot(X_valid,X_train.T)
del X_valid_n
dist.shape

(10000, 290000)

In [16]:
dist+=X_train_n.reshape(1,X_train_n.shape[0])
del X_train_n
dist.shape

(10000, 290000)

In [17]:
distances = np.min(dist,axis=1)
pred_index = np.argmin(dist,axis=1)
distances.shape

(10000,)

In [18]:
pred = y_train[pred_index].copy()
pred[distances>=1] = np.nanmean(y_train)
print("First 10 predictions:", pred[:10] )

First 10 predictions: [ 20.01553  20.01553  20.01553  20.01553 112.15875  20.01553 112.15875
  20.01553 112.15875 112.15875]


<ipython-input-18-cea870104cc1>:2: RuntimeWarning: invalid value encountered in greater_equal
  pred[distances>=1] = np.nanmean(y_train)


In [21]:
m = np.sqrt(np.nanmean( (valid.Price.values-pred)**2 ))
print("Using KNN Regressor - Validation score =",m)

Using KNN Regressor - Validation score = 69.48511
